In [1]:
# dataset
import os
from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset


DATA_DIR = os.environ.get('DATA_DIR')

config = Configuration.create('configs/evtopo-test.yaml')
train, val = loadDataset(config, data_dir=DATA_DIR, device='cpu')

it = iter(val)
tensors = next(it)

for k, v in tensors.items():
	print(f'{k}:\n', v[0].shape)


type:
 torch.Size([19])
staff:
 torch.Size([19])
feature:
 torch.Size([19, 15])
x:
 torch.Size([19])
y1:
 torch.Size([19])
y2:
 torch.Size([19])
matrixH:
 torch.Size([324])
tick:
 torch.Size([19])
division:
 torch.Size([19])
dots:
 torch.Size([19])
beam:
 torch.Size([19])
stemDirection:
 torch.Size([19])
grace:
 torch.Size([19])
timeWarped:
 torch.Size([19])
fullMeasure:
 torch.Size([19])
confidence:
 torch.Size([19])


In [2]:
# RectifySieveJointer
import torch
from starry.topology.models.rectifyJointer import RectifySieveJointer


model = RectifySieveJointer(1, 1)

with torch.no_grad():
	rec, j = model(tensors)

assert tensors['matrixH'][0].shape == j[0].shape

print('types:', tensors['type'].shape)
print('rec:', {k: v[0, 0] for k, v in rec.items()})
print('j:', len(j), j[0].shape)


types: torch.Size([64, 19])
rec: {'tick': tensor(0.8923), 'division': tensor([0.3243, 0.2669, 0.0895, 0.1557, 0.0744, 0.0472, 0.0419]), 'dots': tensor([0.3053, 0.5426, 0.1521]), 'beam': tensor([0.1633, 0.2843, 0.1290, 0.4234]), 'stemDirection': tensor([0.1937, 0.3201, 0.4862]), 'grace': tensor(0.4769), 'timeWarped': tensor(0.4685), 'fullMeasure': tensor(0.7561), 'confidence': tensor(0.5128)}
j: 64 torch.Size([324])


In [3]:
# RectifySieveJointerLoss
import torch
from starry.topology.models.rectifyJointer import RectifySieveJointerLoss


model = RectifySieveJointerLoss(n_trunk_layers=1, n_rectifier_layers=1)

with torch.no_grad():
	loss, metrics = model(tensors)

print('loss:', loss)
print('metrics:', metrics)


loss: tensor(37.5855)
metrics: {'acc_topo': 0.0, 'loss_topo': 2.5409018993377686, 'error_tick': 1081.1961669921875, 'acc_division': 0.0024671051651239395, 'acc_dots': 0.15707236528396606, 'acc_beam': 0.3075657784938812, 'acc_stemDirection': 0.2524670958518982, 'acc_grace': 0.9901315569877625, 'acc_timeWarped': 0.02055921033024788, 'acc_fullMeasure': 0.037828948348760605, 'acc_confidence': 0.046875}
